In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import pandas as pd
import timeit
import time
import tensorflow as tf
from joblib import dump, load
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
def poly_svm(train_data):
    degrees = [1,2,3,4]
    times = []
    regs = []
    for degree in degrees:
        t0 = time.time()
        reg = SVR(kernel="poly", degree = degree)
        reg.fit(train_data[:,:-1], train_data[:,-1])
        t1 = time.time()
        times.append(t1-t0)
        regs.append(reg)
        print("Done degree {}".format(degree))
    return times, regs

def rbf_svm(train_data):
    t0 = time.time()
    reg = SVR(kernel="rbf")
    reg.fit(train_data[:,:-1], train_data[:,-1])
    t1 = time.time()
    return t1-t0, reg

def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

def poly_svm_boost(train_data, M_train_data, data_type):
    degrees = [1,2,3,4]
    times = []
    M_regs = []
    g_regs = []
    for degree in degrees:
        t0 = time.time()
        reg = SVR(kernel="poly", degree = degree)
        reg.fit(M_train_data[:,:-1], M_train_data[:,-1])
        M = reg.predict(train_data[:,:2])
        train_data[:,-1] = train_data[:,-1]/M
        reg_g = SVR(kernel="poly", degree = degree)
        if data_type == 'le':
            reg_g.fit(train_data[:,1:-1], train_data[:,-1])
        elif data_type == 'g':
            reg_g.fit(train_data[:,:-1], train_data[:,-1])
        else:
            raise Exception("Invalid data type")
        t1 = time.time()
        times.append(t1-t0)
        M_regs.append(reg)
        g_regs.append(reg_g)
        print("Done degree {}".format(degree))
    
    return times, M_regs, g_regs

def rbf_svm_boost(train_data, M_train_data, data_type):
    t0 = time.time()
    reg = SVR(kernel="rbf")
    reg.fit(M_train_data[:,:-1], M_train_data[:,-1])
    M = reg.predict(train_data[:,:2])
    train_data[:,-1] = train_data[:,-1]/M
    reg_g = SVR(kernel="rbf")
    if data_type == 'le':
        reg_g.fit(train_data[:,1:-1], train_data[:,-1])
    elif data_type == 'g':
        reg_g.fit(train_data[:,:-1], train_data[:,-1])
    else:
        raise Exception("Invalid data type")
    t1 = time.time()
    return t1-t0, reg, reg_g

def nn_boost(train_data, M_train_data, data_type):
    t0 = time.time()
    _, reg = NN(M_train_data[:,:-1], M_train_data[:,-1])
    M = reg.predict(train_data[:,:2])
    train_data[:,-1] = train_data[:,-1]/np.squeeze(M, axis=-1)
    if data_type == 'le':
        _, reg_g = NN(train_data[:,1:-1], train_data[:,-1])
    elif data_type == 'g':
        _, reg_g = NN(train_data[:,:-1], train_data[:,-1])
    else:
        raise Exception("Invalid data type")
    t1 = time.time()
    return t1-t0, reg, reg_g

In [29]:
data_num = "3a"

# All points

In [ ]:
def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

In [7]:
###################################################################
####################### All dataset ###############################
###################################################################

# Loading dataset
df = pd.read_csv("data/d{}/train/{}.csv".format(data_num, data_num))
df.drop(["c/b", "F"], axis=1, inplace=True)
d = df.to_numpy()

# Poly SVM
times, regs = poly_svm(d)
i = 1
for reg in regs:
    dump(reg, 'results/d{}/models/all/poly_svm_{}.joblib'.format(data_num, i))
    i += 1
np.save("results/d{}/times/all/poly_times.npy".format(data_num), times)

# RBF SVM
times, regs = rbf_svm(d)
dump(regs, 'results/d{}/models/all/rbf_svm.joblib'.format(data_num))
np.save("results/d{}/times/all/rbf_times.npy".format(data_num), times)

# NN
t0 = time.time()
history, model = NN(d[:,:-1], d[:,-1])
t1 = time.time()
model.save("results/d{}/models/all/nn.h5".format(data_num))
np.save("results/d{}/times/all/nn_times.npy".format(data_num), t1-t0)

Done degree 1
Done degree 2
Done degree 3
Done degree 4
Epoch 1/1000
355/355 [==============================] - 1s 2ms/step - loss: 0.3679 - val_loss: 0.0216
Epoch 2/1000
355/355 [==============================] - 1s 1ms/step - loss: 0.0131 - val_loss: 0.0081
Epoch 3/1000
355/355 [==============================] - 0s 1ms/step - loss: 0.0067 - val_loss: 0.0053
Epoch 4/1000
355/355 [==============================] - 0s 1ms/step - loss: 0.0052 - val_loss: 0.0044
Epoch 5/1000
355/355 [==============================] - 1s 2ms/step - loss: 0.0044 - val_loss: 0.0043
Epoch 6/1000
355/355 [==============================] - 0s 1ms/step - loss: 0.0042 - val_loss: 0.0039
Epoch 7/1000
355/355 [==============================] - 1s 2ms/step - loss: 0.0040 - val_loss: 0.0038
Epoch 8/1000
355/355 [==============================] - 1s 1ms/step - loss: 0.0039 - val_loss: 0.0038
Epoch 9/1000
355/355 [==============================] - 0s 1ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 10/1000
355/355 [===

355/355 [==============================] - 0s 1ms/step - loss: 6.1053e-04 - val_loss: 6.3996e-04
Epoch 157/1000
355/355 [==============================] - 0s 955us/step - loss: 6.1127e-04 - val_loss: 6.7972e-04
Epoch 158/1000
355/355 [==============================] - 0s 924us/step - loss: 6.0773e-04 - val_loss: 6.3130e-04
Epoch 159/1000
355/355 [==============================] - 0s 1ms/step - loss: 6.1259e-04 - val_loss: 7.3051e-04
Epoch 160/1000
355/355 [==============================] - 0s 987us/step - loss: 6.0007e-04 - val_loss: 6.1783e-04
Epoch 161/1000
355/355 [==============================] - 0s 1ms/step - loss: 5.9005e-04 - val_loss: 5.6726e-04
Epoch 162/1000
355/355 [==============================] - 0s 1ms/step - loss: 5.7678e-04 - val_loss: 5.9714e-04
Epoch 163/1000
355/355 [==============================] - 0s 908us/step - loss: 5.9058e-04 - val_loss: 5.5767e-04
Epoch 164/1000
355/355 [==============================] - 0s 940us/step - loss: 5.5733e-04 - val_loss: 6.7767e-

# a/c <=1

In [8]:
def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

In [9]:
###################################################################
########################## a/c <= 1 ###############################
###################################################################
#-----------------------------------------------------------------#
######################## NO BOOST #################################
# Loading dataset
d = np.load("data/d{}/train/{}_le.npy".format(data_num, data_num))

# Poly SVM
times, regs = poly_svm(d)
i = 1
for reg in regs:
    dump(reg, 'results/d{}/models/le/poly_svm_{}.joblib'.format(data_num, i))
    i += 1
np.save("results/d{}/times/le/poly_times.npy".format(data_num), times)

# RBF SVM
times, regs = rbf_svm(d)
dump(regs, 'results/d{}/models/le/rbf_svm.joblib'.format(data_num))
np.save("results/d{}/times/le/rbf_times.npy".format(data_num), times)

# NN
t0 = time.time()
history, model = NN(d[:,:-1], d[:,-1])
t1 = time.time()
model.save("results/d{}/models/le/nn.h5".format(data_num))
np.save("results/d{}/times/le/nn_times.npy".format(data_num), t1-t0)


Done degree 1
Done degree 2
Done degree 3
Done degree 4
Epoch 1/1000
221/221 [==============================] - 1s 1ms/step - loss: 0.0963 - val_loss: 0.0166
Epoch 2/1000
221/221 [==============================] - 0s 926us/step - loss: 0.0101 - val_loss: 0.0068
Epoch 3/1000
221/221 [==============================] - 0s 1ms/step - loss: 0.0058 - val_loss: 0.0046
Epoch 4/1000
221/221 [==============================] - 0s 1ms/step - loss: 0.0041 - val_loss: 0.0036
Epoch 5/1000
221/221 [==============================] - 0s 1ms/step - loss: 0.0033 - val_loss: 0.0029
Epoch 6/1000
221/221 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 7/1000
221/221 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0021
Epoch 8/1000
221/221 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0019
Epoch 9/1000
221/221 [==============================] - 0s 2ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 10/1000
221/221 [=

Epoch 76/1000
221/221 [==============================] - 0s 1ms/step - loss: 6.2380e-04 - val_loss: 5.6806e-04
Epoch 77/1000
221/221 [==============================] - 0s 1ms/step - loss: 6.1981e-04 - val_loss: 6.5096e-04
Epoch 78/1000
221/221 [==============================] - 0s 1ms/step - loss: 6.0990e-04 - val_loss: 5.6889e-04
Epoch 79/1000
221/221 [==============================] - 0s 1ms/step - loss: 5.9418e-04 - val_loss: 5.7570e-04
Epoch 80/1000
221/221 [==============================] - 0s 986us/step - loss: 6.0968e-04 - val_loss: 5.9010e-04
Epoch 81/1000
221/221 [==============================] - 0s 1ms/step - loss: 5.8987e-04 - val_loss: 5.1049e-04
Epoch 82/1000
221/221 [==============================] - 0s 1ms/step - loss: 5.8323e-04 - val_loss: 7.1245e-04
Epoch 83/1000
221/221 [==============================] - 0s 1ms/step - loss: 6.2540e-04 - val_loss: 5.0206e-04
Epoch 84/1000
221/221 [==============================] - 0s 992us/step - loss: 5.8945e-04 - val_loss: 5.5394e-

In [18]:
def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

def nn_boost(train_data, M_train_data, data_type):
    t0 = time.time()
    _, reg = NN(M_train_data[:,:-1], M_train_data[:,-1])
    M = reg.predict(train_data[:,:2])
    train_data[:,-1] = train_data[:,-1]/np.squeeze(M, axis=-1)
    if data_type == 'le':
        _, reg_g = NN(train_data[:,1:-1], train_data[:,-1])
    elif data_type == 'g':
        _, reg_g = NN(train_data[:,:-1], train_data[:,-1])
    else:
        raise Exception("Invalid data type")
    t1 = time.time()
    return t1-t0, reg, reg_g

In [19]:
#-----------------------------------------------------------------#
########################### BOOST #################################
# Generating dataset
# Loading dataset
d = np.load("data/d{}/train/{}_le.npy".format(data_num, data_num))

data = []
for row in d:
    if (row[2] > 1.5 and row[2] < 1.65):
        data.append(np.delete(row, 2))
M_data = np.array(data)
'''
# Poly SVM
times, M_regs, g_regs = poly_svm_boost(d, M_data, "le")
for i in range(4):
    dump(M_regs[i], 'results/d{}/models/le/boost/M_poly_svm_{}.joblib'.format(data_num,i))
    dump(g_regs[i], 'results/d{}/models/le/boost/g_poly_svm_{}.joblib'.format(data_num,i))
np.save("results/d{}/times/le/poly_times_boost.npy".format(data_num), times)

# RBF SVM
times, M_regs, g_regs = rbf_svm_boost(d, M_data, "le")
dump(M_regs, 'results/d{}/models/le/boost/M_rbf_svm.joblib'.format(data_num))
dump(g_regs, 'results/d{}/models/le/boost/g_rbf_svm.joblib'.format(data_num))
np.save("results/d{}/times/le/rbf_times_boost.npy".format(data_num), times)
'''
# NN
times, M_regs, g_regs = nn_boost(d, M_data, "le")
M_regs.save("results/d{}/models/le/boost/M_nn.h5".format(data_num))
g_regs.save("results/d{}/models/le/boost/g_nn.h5".format(data_num))
np.save("results/d{}/times/le/nn_times_boost.npy".format(data_num), times)

Epoch 1/1000
11/11 [==============================] - 0s 10ms/step - loss: 1.3292 - val_loss: 1.3011
Epoch 2/1000
11/11 [==============================] - 0s 3ms/step - loss: 1.1834 - val_loss: 1.1210
Epoch 3/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.9918 - val_loss: 0.9027
Epoch 4/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.7673 - val_loss: 0.6488
Epoch 5/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.5024 - val_loss: 0.3618
Epoch 6/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.2469 - val_loss: 0.1413
Epoch 7/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.0794 - val_loss: 0.0335
Epoch 8/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.0197 - val_loss: 0.0179
Epoch 9/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.0199 - val_loss: 0.0195
Epoch 10/1000
11/11 [==============================] - 0s 3ms/step - loss: 0.0182 - val_loss: 0.016

11/11 [==============================] - 0s 3ms/step - loss: 7.5810e-04 - val_loss: 7.3213e-04
Epoch 160/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.5335e-04 - val_loss: 7.0634e-04
Epoch 161/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.5101e-04 - val_loss: 6.9212e-04
Epoch 162/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.4931e-04 - val_loss: 6.9998e-04
Epoch 163/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.4207e-04 - val_loss: 6.8124e-04
Epoch 164/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.3056e-04 - val_loss: 6.8342e-04
Epoch 165/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.2928e-04 - val_loss: 6.9576e-04
Epoch 166/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.5484e-04 - val_loss: 6.6541e-04
Epoch 167/1000
11/11 [==============================] - 0s 3ms/step - loss: 7.2352e-04 - val_loss: 6.9069e-04
Epoch 168/1000
11/11 [===

11/11 [==============================] - 0s 3ms/step - loss: 2.8525e-04 - val_loss: 2.5026e-04
Epoch 308/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.8199e-04 - val_loss: 2.3713e-04
Epoch 309/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.7746e-04 - val_loss: 2.4948e-04
Epoch 310/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.7336e-04 - val_loss: 2.4008e-04
Epoch 311/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.7861e-04 - val_loss: 2.2343e-04
Epoch 312/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.8442e-04 - val_loss: 2.3461e-04
Epoch 313/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.8636e-04 - val_loss: 2.1390e-04
Epoch 314/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.6569e-04 - val_loss: 2.3336e-04
Epoch 315/1000
11/11 [==============================] - 0s 3ms/step - loss: 2.6264e-04 - val_loss: 2.3071e-04
Epoch 316/1000
11/11 [===

221/221 [==============================] - 0s 1ms/step - loss: 6.8547e-04 - val_loss: 6.5407e-04


# a/c > 1

In [34]:
def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

def nn_boost(train_data, M_train_data, data_type):
    t0 = time.time()
    _, reg = NN(M_train_data[:,:-1], M_train_data[:,-1])
    M = reg.predict(train_data[:,:2])
    train_data[:,-1] = train_data[:,-1]/np.squeeze(M, axis=-1)
    if data_type == 'le':
        _, reg_g = NN(train_data[:,1:-1], train_data[:,-1])
    elif data_type == 'g':
        _, reg_g = NN(train_data[:,:-1], train_data[:,-1])
    else:
        raise Exception("Invalid data type")
    t1 = time.time()
    return t1-t0, reg, reg_g

In [35]:
###################################################################
########################## a/c > 1 ################################
###################################################################
#-----------------------------------------------------------------#
######################## NO BOOST #################################
# Loading dataset
d = np.load("data/d{}/train/{}_g.npy".format(data_num, data_num))
'''
# Poly SVM
times, regs = poly_svm(d)
i = 1
for reg in regs:
    dump(reg, 'results/d{}/models/g/poly_svm_{}.joblib'.format(data_num, i))
    i += 1
np.save("results/d{}/times/g/poly_times.npy".format(data_num), times)

# RBF SVM
times, regs = rbf_svm(d)
dump(regs, 'results/d{}/models/g/rbf_svm.joblib'.format(data_num))
np.save("results/d{}/times/g/rbf_times.npy".format(data_num), times)
'''
# NN
t0 = time.time()
history, model = NN(d[:,:-1], d[:,-1])
t1 = time.time()
model.save("results/d{}/models/g/nn.h5".format(data_num))
np.save("results/d{}/times/g/nn_times.npy".format(data_num), t1-t0)

Epoch 1/1000
1236/1236 [==============================] - 2s 1ms/step - loss: 0.0121 - val_loss: 5.9273e-04
Epoch 2/1000
1236/1236 [==============================] - 1s 982us/step - loss: 4.7989e-04 - val_loss: 3.7607e-04
Epoch 3/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 3.7419e-04 - val_loss: 4.0987e-04
Epoch 4/1000
1236/1236 [==============================] - 1s 939us/step - loss: 2.9948e-04 - val_loss: 3.4210e-04
Epoch 5/1000
1236/1236 [==============================] - 2s 1ms/step - loss: 2.5270e-04 - val_loss: 2.5527e-04
Epoch 6/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 2.1511e-04 - val_loss: 1.8231e-04
Epoch 7/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 1.8630e-04 - val_loss: 1.8397e-04
Epoch 8/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 1.7167e-04 - val_loss: 1.4602e-04
Epoch 9/1000
1236/1236 [==============================] - 2s 1ms/step - loss: 1.5862e-04 - val_loss: 1.3

In [36]:
def NN(X_train, y_train):
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X_train.shape[1]
    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=[early_stopping_monitor])
    return history, model

def nn_boost(train_data, M_train_data, data_type):
    t0 = time.time()
    _, reg = NN(M_train_data[:,:-1], M_train_data[:,-1])
    M = reg.predict(train_data[:,:2])
    train_data[:,-1] = train_data[:,-1]/np.squeeze(M, axis=-1)
    if data_type == 'le':
        _, reg_g = NN(train_data[:,1:-1], train_data[:,-1])
    elif data_type == 'g':
        _, reg_g = NN(train_data[:,:-1], train_data[:,-1])
    else:
        raise Exception("Invalid data type")
    t1 = time.time()
    return t1-t0, reg, reg_g

In [37]:
#-----------------------------------------------------------------#
########################### BOOST #################################
# Generating dataset
# Loading dataset
d = np.load("data/d{}/train/{}_g.npy".format(data_num, data_num))

data = []
for row in d:
    if (row[2] > 1.5 and row[2] < 1.65):
        data.append(np.delete(row, 2))
M_data = np.array(data)
'''
# Poly SVM
times, M_regs, g_regs = poly_svm_boost(d, M_data, "g")
for i in range(4):
    dump(M_regs[i], 'results/d{}/models/g/boost/M_poly_svm_{}.joblib'.format(data_num,i))
    dump(g_regs[i], 'results/d{}/models/g/boost/g_poly_svm_{}.joblib'.format(data_num,i))
np.save("results/d{}/times/g/poly_times_boost.npy".format(data_num), times)

# RBF SVM
times, M_regs, g_regs = rbf_svm_boost(d, M_data, "g")
dump(M_regs, 'results/d{}/models/g/boost/M_rbf_svm.joblib'.format(data_num))
dump(g_regs, 'results/d{}/models/g/boost/g_rbf_svm.joblib'.format(data_num))
np.save("results/d{}/times/g/rbf_times_boost.npy".format(data_num), times)
'''
# NN
times, M_regs, g_regs = nn_boost(d, M_data, "g")
M_regs.save("results/d{}/models/g/boost/M_nn.h5".format(data_num))
g_regs.save("results/d{}/models/g/boost/g_nn.h5".format(data_num))
np.save("results/d{}/times/g/nn_times_boost.npy".format(data_num), times)

Epoch 1/1000
46/46 [==============================] - 1s 3ms/step - loss: 0.2978 - val_loss: 0.0865
Epoch 2/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0477 - val_loss: 0.0352
Epoch 3/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.0354 - val_loss: 0.0326
Epoch 4/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0330 - val_loss: 0.0303
Epoch 5/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.0303 - val_loss: 0.0275
Epoch 6/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0276 - val_loss: 0.0250
Epoch 7/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0250 - val_loss: 0.0227
Epoch 8/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0228 - val_loss: 0.0209
Epoch 9/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.0209 - val_loss: 0.0192
Epoch 10/1000
46/46 [==============================] - 0s 1ms/step - loss: 0.0192 - val_loss: 0.0177

Epoch 78/1000
46/46 [==============================] - 0s 2ms/step - loss: 8.0302e-05 - val_loss: 7.5443e-05
Epoch 79/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.3790e-05 - val_loss: 7.7307e-05
Epoch 80/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.6214e-05 - val_loss: 8.2217e-05
Epoch 81/1000
46/46 [==============================] - 0s 2ms/step - loss: 8.7232e-05 - val_loss: 7.2868e-05
Epoch 82/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.6267e-05 - val_loss: 8.4921e-05
Epoch 83/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.8692e-05 - val_loss: 7.2122e-05
Epoch 84/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.3798e-05 - val_loss: 7.7281e-05
Epoch 85/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.0197e-05 - val_loss: 6.7804e-05
Epoch 86/1000
46/46 [==============================] - 0s 2ms/step - loss: 7.2580e-05 - val_loss: 7.2780e-05
Epoch 87/1000
46/46

1236/1236 [==============================] - 1s 1ms/step - loss: 8.3374e-05 - val_loss: 7.4199e-05
Epoch 32/1000
1236/1236 [==============================] - 1s 951us/step - loss: 7.9977e-05 - val_loss: 7.4712e-05
Epoch 33/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 8.1578e-05 - val_loss: 1.3107e-04
Epoch 34/1000
1236/1236 [==============================] - 1s 986us/step - loss: 8.3514e-05 - val_loss: 7.3017e-05
Epoch 35/1000
1236/1236 [==============================] - 1s 999us/step - loss: 7.9338e-05 - val_loss: 9.1514e-05
Epoch 36/1000
1236/1236 [==============================] - 1s 956us/step - loss: 8.2733e-05 - val_loss: 8.3715e-05
Epoch 37/1000
1236/1236 [==============================] - 1s 983us/step - loss: 8.0852e-05 - val_loss: 7.5281e-05
Epoch 38/1000
1236/1236 [==============================] - 1s 1ms/step - loss: 7.9499e-05 - val_loss: 8.9711e-05
Epoch 39/1000
1236/1236 [==============================] - 1s 968us/step - loss: 7.8828e-05 - val_lo